<a href="https://colab.research.google.com/github/SadbinShakil/Scrapping-amazon-Data/blob/main/scraping_Amazon_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#it will install beautiful soup in your PC
!pip install bs4

In [ ]:
#Cz we will send request from our local pc to amazon website
!pip install requests

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [ ]:
URL = "https://www.amazon.in/s?k=ps5&crid=PLFTP7YZE04E&sprefix=ps%2Caps%2C582&ref=nb_sb_noss_2"
#User agent from what is my user agent website
HEADERS = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,'referer':'https://www.google.com/'}

In [ ]:
#As we already declared the header, Now we will be making request for the available site
webpage = requests.get(URL,headers=HEADERS)

In [ ]:
#Now check wheather I got the response (200) or not
webpage

<Response [200]>

In [ ]:
#seeing the content of that webpage
webpage.content

In [ ]:
type(webpage.content)

bytes

In [ ]:
#so its in the byte format and we need it in the HTML format. so we will use soup
soup = BeautifulSoup(webpage.content, "html.parser")
soup

In [ ]:
#now this is on proper html format
type(soup)

bs4.BeautifulSoup

In [ ]:
All_links = soup.find_all("a", attrs={'class': 'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold'})

In [ ]:
All_links[0]

<a class="a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-bold" href="/sspa/click?ie=UTF8&amp;spc=MToxNjcwODM2ODcwNjMzMDMxOjE3MDY3NjI1NDk6c3BfYXRmOjMwMDEwNzgyNTkyMTUzMjo6MDo6&amp;url=%2FPSS-Playstation-Anti-Scratch-Protective-Replacement%2Fdp%2FB0CJCC1BYK%2Fref%3Dsr_1_1_sspa%3Fcrid%3DPLFTP7YZE04E%26keywords%3Dps5%26qid%3D1706762549%26sprefix%3Dps%252Caps%252C582%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1">PlayStation 5</a>

In [ ]:
# get only the link from a tag
link = All_links[1].get('href')

In [ ]:
#we are getting the half link and we need to append something with it
product_link = "https://amazon.in" + link

In [ ]:
product_link

'https://amazon.in/Sony-PS5-Digital-Standalone/dp/B0BSNHFVL4/ref=sr_1_2?crid=PLFTP7YZE04E&keywords=ps5&qid=1706762549&sprefix=ps%2Caps%2C582&sr=8-2'

In [ ]:
#now from that new webpage, lets find some info
webpage = requests.get(product_link,headers=HEADERS)
webpage

<Response [200]>

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
soup

In [ ]:
soup.find("h1", attrs={'id': 'title'})

<h1 class="a-size-large a-spacing-none" id="title"> <span class="a-size-large product-title-word-break" id="productTitle">        Sony PS5 Console Digital Standalone       </span> </h1>

In [ ]:
#only to get the text
soup.find("h1", attrs={'id': 'title'}).text

'         Sony PS5 Console Digital Standalone        '

In [ ]:
#Remove empty spaces
soup.find("h1", attrs={'id': 'title'}).text.strip()

'Sony PS5 Console Digital Standalone'

In [ ]:
#find the price
soup.find("span", attrs={'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'})

<span class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay" data-a-color="base" data-a-size="xl"><span class="a-offscreen">₹44,987</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">44,987</span></span></span>

In [ ]:
soup.find("span", attrs={'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).text

'₹44,987₹44,987'

In [ ]:
#As we are getting hidden plus actual price, I need to remove the hidden price
soup.find("span", attrs={'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs={"class": "a-offscreen"}).text

'₹44,987'

## ## **As we already did all the things part by part, now lets make the code meaningful:**

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=ps5&crid=VC1ESD53NDYB&sprefix=p%2Caps%2C806&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df.head()

,title,price,rating,reviews,availability
0,PSS PS5 Faceplate for Playstation 5 Disc Editi...,,5.0 out of 5 stars,4 ratings,In stock
1,Sony PS5 Console Digital Standalone,,4.7 out of 5 stars,473 ratings,In stock
2,Sony PS5 Console,,4.6 out of 5 stars,"1,340 ratings",In stock
3,Sony PS5 Console - Call of Duty Modern Warfare...,,4.7 out of 5 stars,33 ratings,In stock
4,Sony PS5 Console Standard DualSense bundle,,4.6 out of 5 stars,79 ratings,In stock
